In [123]:
import pandas as pd
from tqdm import tqdm
import warnings
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor

warnings.filterwarnings('ignore')

In [124]:
df = pd.read_csv("files\\pred_final.csv")
df

,obj_prg,obj_subprg,obj_key,Кодзадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,Статуспоэкспертизе,Экспертиза,date_report,season,Скорость,Кол-во рабочих,Генподрядчик,Генпроектировщик,Время на выполнение,Площадь
0,0,0,022-0527,1,0,2022-01-14,2023-03-30,0,0,2023-01-17,1,0.257576,0.0,0.0,0.0,440,9850.0
1,0,0,022-0354,1,0,2022-12-01,2023-07-27,0,0,2023-01-17,1,0.462121,0.0,0.0,0.0,238,20271.0
2,0,1,022-0513,1,0,2023-01-23,2023-09-29,0,0,2023-01-17,1,0.416667,0.0,0.0,0.0,249,6892.0
3,0,1,020-0684,1,0,2020-11-03,2022-02-01,0,0,2023-01-17,4,0.000000,0.0,2.0,1.0,455,2488.6
4,0,1,019-0589,1,0,2020-11-03,2022-05-16,0,0,2023-01-17,4,0.000000,0.0,2.0,1.0,559,3751.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35196,1,2,019-0477,7.4,93,2020-04-28,2023-05-31,0,0,2023-05-29,2,0.704545,5.0,12.0,16.0,1128,0.0
35197,0,0,020-0710,8,0,2023-08-31,2024-04-10,0,0,2023-05-29,3,0.000000,156.0,5.0,8.0,223,0.0
35198,0,0,019-0675,8,0,2023-09-01,2024-04-11,0,0,2023-05-29,4,0.000000,139.0,22.0,33.0,223,0.0
35199,0,1,020-0712,8,0,2023-06-19,2024-01-26,0,0,2023-05-29,2,0.000000,53.0,5.0,8.0,221,0.0


In [125]:
date_reports = df['date_report'].unique()
len(date_reports)
# todo Добавить площадщь
columns = ['obj_key', 'obj_subprg', 'Кодзадачи', 'ДатаНачалаЗадачи', 'Статуспоэкспертизе', 'Экспертиза', 'season',
           'Скорость', 'Кол-во рабочих', 'Генподрядчик', 'Генпроектировщик', 'Площадь', *date_reports, 'y']
columns

['obj_key',
 'obj_subprg',
 'Кодзадачи',
 'ДатаНачалаЗадачи',
 'Статуспоэкспертизе',
 'Экспертиза',
 'season',
 'Скорость',
 'Кол-во рабочих',
 'Генподрядчик',
 'Генпроектировщик',
 'Площадь',
 '2023-01-17',
 '2023-01-23',
 '2023-02-06',
 '2023-02-13',
 '2023-02-20',
 '2023-02-27',
 '2023-03-06',
 '2023-03-13',
 '2023-03-20',
 '2023-03-27',
 '2023-04-03',
 '2023-04-10',
 '2023-04-17',
 '2023-04-24',
 '2023-05-02',
 '2023-05-10',
 '2023-05-15',
 '2023-05-22',
 '2023-05-29',
 'y']

In [126]:
df = df.sort_values(['obj_key', 'Кодзадачи', 'ПроцентЗавершенияЗадачи', 'date_report'])

In [127]:
df[df['obj_key'] == '017-0520'].head(1).values

array([[0, 1, '017-0520', '1', 100, '2018-11-01', '2022-02-15', 0, 0,
        '2023-01-17', 4, 0.7575757575757576, 0.0, 5.0, 8.0, 1202, 6367.0]],
      dtype=object)

In [128]:
len(date_reports)

19

In [129]:
d_dates = {key: np.nan for key in date_reports}
list(d_dates.values())

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]

In [130]:
new_df = pd.DataFrame(columns=columns)
keys = df['obj_key'].unique()
keys.sort()
# pd.Series(row_data, index=columns
for key in tqdm(keys):
    codes = df[df['obj_key'] == key]['Кодзадачи'].unique()
    for code in codes:
        objects = df[(df['obj_key'] == key) & (df['Кодзадачи'] == code)]
        start = objects.head(1)
        last = objects.tail(1)
        obj_key = key
        obj_subprg = last['obj_subprg'].values[0]
        obj_code = code
        start_date = start['ДатаНачалаЗадачи'].values[0]
        status_exp = last['Статуспоэкспертизе'].values[0]
        exp = last['Экспертиза'].values[0]
        season = last['season'].values[0]
        speed = last['Скорость'].values[0]
        rab_count = start['Кол-во рабочих'].values[0]
        gen_pod = last['Генподрядчик'].values[0]
        gen_proc = last['Генпроектировщик'].values[0]
        square = last['Площадь'].values[0]
        y_value = last['Время на выполнение'].values[0]
        dates = objects['date_report'].unique()
        d_dates = {key: np.nan for key in date_reports}
        row_data = [obj_key, obj_subprg, obj_code, start_date, status_exp, exp, season, speed, rab_count, gen_pod,
                    gen_proc, square]
        for date in dates:
            prc = objects[objects['date_report'] == date].head(1)['ПроцентЗавершенияЗадачи'].values[0]
            d_dates[date] = prc
        row_data.extend(list(d_dates.values()))
        row_data.append(y_value)
        new_df = new_df.append(pd.Series(row_data, index=columns), ignore_index=True)
new_df

100%|██████████| 72/72 [00:19<00:00,  3.71it/s]


,obj_key,obj_subprg,Кодзадачи,ДатаНачалаЗадачи,Статуспоэкспертизе,Экспертиза,season,Скорость,Кол-во рабочих,Генподрядчик,...,2023-04-03,2023-04-10,2023-04-17,2023-04-24,2023-05-02,2023-05-10,2023-05-15,2023-05-22,2023-05-29,y
0,017-0520,1,1,2018-11-01,0,0,4,0.757576,0.0,5.0,...,100,100,100,100,100,100,100,100,100,1202
1,017-0520,1,1.4.2,2021-12-15,0,0,1,0.757576,0.0,5.0,...,100,100,100,100,100,100,100,100,100,62
2,017-0520,1,3,2021-02-20,0,0,2,0.083333,0.0,5.0,...,11,11,11,11,11,11,11,11,11,373
3,017-0520,1,3.1,2021-02-20,0,0,2,0.681818,0.0,5.0,...,38,38,38,38,38,38,38,38,38,220
4,017-0520,1,3.11,2021-04-20,0,0,2,0.000000,0.0,5.0,...,0,0,0,0,0,0,0,0,0,314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,022-0631,3,5.11,2024-10-01,0,0,4,0.000000,0.0,2.0,...,0,0,0,0,0,0,0,0,0,4
1841,022-0631,3,7.1,2024-10-01,0,0,3,0.000000,0.0,2.0,...,0,0,0,0,0,0,0,0,0,44
1842,022-0631,3,7.1.8,2024-11-11,0,0,4,0.000000,0.0,2.0,...,0,0,0,0,0,0,0,0,0,3
1843,022-0631,3,7.4,2024-11-12,0,0,4,0.000000,0.0,2.0,...,0,0,0,0,0,0,0,0,0,28


In [131]:
def find_good_idx(vals: np.ndarray, start_index: int = 12) -> tuple[int, bool]:
    good_idx = 0
    for i in range(start_index, len(vals) - 2):
        if not np.isnan(vals[i]):
            good_idx = i
    if good_idx == 0:
        return 0, False
    else:
        return good_idx, True

In [132]:
for i, row in new_df.iterrows():
    has_nan = row.isna().any()
    if has_nan:
        vals = row.values
        for j in range(12, len(vals) - 1):
            if np.isnan(vals[j]):
                good_idx, ans = find_good_idx(vals, i)
                if ans:
                    vals[j] = vals[good_idx]
                else:
                    good_idx, ans = find_good_idx(vals, 12)
                    vals[j] = vals[good_idx]
        new_df.loc[i] = vals

In [133]:
new_df

,obj_key,obj_subprg,Кодзадачи,ДатаНачалаЗадачи,Статуспоэкспертизе,Экспертиза,season,Скорость,Кол-во рабочих,Генподрядчик,...,2023-04-03,2023-04-10,2023-04-17,2023-04-24,2023-05-02,2023-05-10,2023-05-15,2023-05-22,2023-05-29,y
0,017-0520,1,1,2018-11-01,0,0,4,0.757576,0.0,5.0,...,100,100,100,100,100,100,100,100,100,1202
1,017-0520,1,1.4.2,2021-12-15,0,0,1,0.757576,0.0,5.0,...,100,100,100,100,100,100,100,100,100,62
2,017-0520,1,3,2021-02-20,0,0,2,0.083333,0.0,5.0,...,11,11,11,11,11,11,11,11,11,373
3,017-0520,1,3.1,2021-02-20,0,0,2,0.681818,0.0,5.0,...,38,38,38,38,38,38,38,38,38,220
4,017-0520,1,3.11,2021-04-20,0,0,2,0.000000,0.0,5.0,...,0,0,0,0,0,0,0,0,0,314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,022-0631,3,5.11,2024-10-01,0,0,4,0.000000,0.0,2.0,...,0,0,0,0,0,0,0,0,0,4
1841,022-0631,3,7.1,2024-10-01,0,0,3,0.000000,0.0,2.0,...,0,0,0,0,0,0,0,0,0,44
1842,022-0631,3,7.1.8,2024-11-11,0,0,4,0.000000,0.0,2.0,...,0,0,0,0,0,0,0,0,0,3
1843,022-0631,3,7.4,2024-11-12,0,0,4,0.000000,0.0,2.0,...,0,0,0,0,0,0,0,0,0,28


In [134]:
new_df['Генпроектировщик'] = new_df['Генпроектировщик'].astype(int)
new_df['Генподрядчик'] = new_df['Генподрядчик'].astype(int)
new_df['Кол-во рабочих'] = new_df['Кол-во рабочих'].astype(int)
new_df['ДатаНачалаЗадачи'] = pd.to_datetime(new_df['ДатаНачалаЗадачи']).apply(lambda x: x.toordinal())
new_df

,obj_key,obj_subprg,Кодзадачи,ДатаНачалаЗадачи,Статуспоэкспертизе,Экспертиза,season,Скорость,Кол-во рабочих,Генподрядчик,...,2023-04-03,2023-04-10,2023-04-17,2023-04-24,2023-05-02,2023-05-10,2023-05-15,2023-05-22,2023-05-29,y
0,017-0520,1,1,736999,0,0,4,0.757576,0,5,...,100,100,100,100,100,100,100,100,100,1202
1,017-0520,1,1.4.2,738139,0,0,1,0.757576,0,5,...,100,100,100,100,100,100,100,100,100,62
2,017-0520,1,3,737841,0,0,2,0.083333,0,5,...,11,11,11,11,11,11,11,11,11,373
3,017-0520,1,3.1,737841,0,0,2,0.681818,0,5,...,38,38,38,38,38,38,38,38,38,220
4,017-0520,1,3.11,737900,0,0,2,0.000000,0,5,...,0,0,0,0,0,0,0,0,0,314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,022-0631,3,5.11,739160,0,0,4,0.000000,0,2,...,0,0,0,0,0,0,0,0,0,4
1841,022-0631,3,7.1,739160,0,0,3,0.000000,0,2,...,0,0,0,0,0,0,0,0,0,44
1842,022-0631,3,7.1.8,739201,0,0,4,0.000000,0,2,...,0,0,0,0,0,0,0,0,0,3
1843,022-0631,3,7.4,739202,0,0,4,0.000000,0,2,...,0,0,0,0,0,0,0,0,0,28


In [135]:
cat_features = ["Кодзадачи", 'obj_subprg', 'Статуспоэкспертизе', 'Экспертиза', 'season', 'Генподрядчик',
                'Генпроектировщик']

In [136]:
x_train, x_test, y_train, y_test = train_test_split(new_df.drop(['obj_key', 'y'], axis=1), new_df['y'], test_size=0.2,
                                                    random_state=228)

In [141]:
# leaf_estimation_backtracking="AnyImprovement"
model = CatBoostRegressor(iterations=2000, random_seed=18, loss_function='RMSE', learning_rate=0.05, max_depth=9, early_stopping_rounds=700, eval_metric="MAE")

In [142]:
model.fit(x_train, y_train, use_best_model=True, cat_features=cat_features, eval_set=(x_test, y_test))

0:	learn: 189.8880711	test: 187.7815832	best: 187.7815832 (0)	total: 42.5ms	remaining: 1m 24s
1:	learn: 184.4988981	test: 182.5323411	best: 182.5323411 (1)	total: 89.4ms	remaining: 1m 29s
2:	learn: 179.5142609	test: 177.1462187	best: 177.1462187 (2)	total: 135ms	remaining: 1m 29s
3:	learn: 174.9292867	test: 171.9822701	best: 171.9822701 (3)	total: 179ms	remaining: 1m 29s
4:	learn: 170.6592602	test: 167.5121780	best: 167.5121780 (4)	total: 224ms	remaining: 1m 29s
5:	learn: 166.3537674	test: 163.0798630	best: 163.0798630 (5)	total: 269ms	remaining: 1m 29s
6:	learn: 162.4036986	test: 159.0705245	best: 159.0705245 (6)	total: 319ms	remaining: 1m 30s
7:	learn: 158.6829548	test: 155.5438647	best: 155.5438647 (7)	total: 370ms	remaining: 1m 32s
8:	learn: 155.6164865	test: 152.6471875	best: 152.6471875 (8)	total: 421ms	remaining: 1m 33s
9:	learn: 152.0996822	test: 148.9766320	best: 148.9766320 (9)	total: 464ms	remaining: 1m 32s
10:	learn: 148.8233697	test: 145.6577427	best: 145.6577427 (10)	tota

In [143]:
model.get_feature_importance(prettified=True)

,Feature Id,Importances
0,Кодзадачи,26.509957
1,ДатаНачалаЗадачи,25.243225
2,obj_subprg,6.812297
3,season,6.666858
4,Генпроектировщик,5.746250
5,Генподрядчик,4.748267
6,Статуспоэкспертизе,3.125974
7,2023-01-23,2.859704
8,Кол-во рабочих,2.538713
9,2023-02-20,1.662882


In [151]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, model.predict(x_test))

72.86046055154304

In [148]:
vals = new_df['y'].unique()
vals.sort()
print(min(vals), max(vals))


0 1547


In [153]:
len(new_df)

1845

In [152]:
new_df.to_csv("files\\times.csv", index=False)